# Costweet : CNN + Fasttext

Ce notebook classe des phrases en 3 catégories "cher", "moyen","éco".

Il fait partie d'une série de notebook permettant de comparer différentes approches de classification en faisant varier les architectures de réseaux de neurones et les méthodes d'encodification des textes.

Il s'appuie sur un ensemble de données de petite taille.

Cet ensemble est vectorisé en utilisant la librairie fasttext de facebook (camdial.org).

Le réseau de neurones est architecturé de trois couches 2 de type convolution. Ce réseau est identique à celui utilisé par le notebook intitulé "Costweet : CNN + manuel encoding"

L'accuracy obtenu le 7 février 2018  est de 92,7% à comparer aux 85% obtenus avec avec un réseau sans convolution.

Keras uses tensorflow uses sklearn uses numpy and pandas

Ce notebook s'inspire fortement du code de François Chollet https://github.com/keras-team/keras/blob/master/examples/pretrained_word_embeddings.py

In [1]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Embedding, Conv1D, Input, GlobalMaxPooling1D, MaxPooling1D
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import text_to_word_sequence
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Model
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
#for fasttext
from __future__ import print_function
from gensim.models import KeyedVectors






Using TensorFlow backend.
d:\code\ml\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


 DATA PROCESS
 les données ont été saisies sous excel et sauvegardées en csv utf-8 bizarement avec l'option séparateur virgules de microsoft
 


In [2]:
def getDataset():
	dataset = pd.read_csv("RuleCout3.csv", delimiter=";", encoding='utf-8')
	#print ( 'dataset loaded with shape',dataset.shape)
	X = dataset['text']
	Y = dataset['class']
	dataset.set_index('text') #pour avoir les classes dans un ordre quelconque
	return dataset, X, Y

# utile pour les colonnes sur une ligne et afficher toutes les lignes
pd.set_option('display.width', 200)
pd.set_option('display.max_rows', 1000)

# on ajoutera peu à peu de nouvelles colonnes au dataset 
dataset, X, Y = getDataset()
print(dataset)

                              text  class
0            à la bonne franquette    éco
1                  à prix modique     éco
2          aucun problème d'argent   cher
3                      avantageux     éco
4                    bas de gamme     éco
5                         bas prix    éco
6                bistrot pas cher     éco
7                              bon   cher
8                     bon japonais   cher
9                      bon marché     éco
10       bon rapport qualité prix   moyen
11                         burger     éco
12                   burger classe  moyen
13                         cantine    éco
14                         caviar    cher
15          ce n'est pas important  moyen
16           cent euros pour deux    cher
17            c'est la fin du mois    éco
18                           cher    cher
19             dix euros pour deux    éco
20                             éco    éco
21                     économique     éco
22                  entre 10 et 20

# Chargement de fasttext
Fasttext needs 10 minutes to be loaded
C'est le temps de chargement du dataset fasttextwiki.fr.vec donne un vocabulaire de 1 115 449 mots et donc autant de vecteurs.

Il vaudrait mieux charger avecle .bin ce serait plus rapide. Le .bin pèse 5 gigas-octets !

Ces vecteurs ont été valorisé en utilisant l'approche skip-gram décrite dans Bojanowski et al. (2016) en utilisant les paramètres par défaut.

La dimension de ces vesteurs est de 300.

We are publishing pre-trained word vectors for 294 languages, trained on Wikipedia using fastText.
These vectors in dimension 300 were obtained using the skip-gram model described in Bojanowski et al. (2016) with default parameters. 

origine #https://blog.manash.me/how-to-use-pre-trained-word-vectors-from-facebooks-fasttext-a71e6d55f27





In [3]:
# Creating the model
fr_model = KeyedVectors.load_word2vec_format('../data/fasttextwiki.fr.vec')

# Getting the tokens 
words = []
for word in fr_model.vocab:
    words.append(word)

# Printing out number of tokens available
print("Number of Tokens: {}".format(len(words)))





Number of Tokens: 1152449


In [4]:
# encode retourne un vecteur en fonction d'un mot
def score(word):
    return  fr_model[word]

vector_troquet = score('troquet')

print ( 'score troquet', vector_troquet, len(vector_troquet))


score troquet [-0.28589    -0.26518     0.076356   -0.017638    0.20075    -0.037748
 -0.20504     0.22178    -0.16894     0.4035     -0.34198     0.037411
 -0.081518   -0.1229     -0.51538     0.0088103   0.084543   -0.082374
 -0.38133    -0.07603     0.10483    -0.019706   -0.04029     0.37365
 -0.37959    -0.54548    -0.31914    -0.31262    -0.060444   -0.053712
  0.43251    -0.1079      0.23003    -0.066676   -0.29099    -0.081674
 -0.20116    -0.32752     0.091499    0.081911    0.051361    0.076685
 -0.071495   -0.070764    0.28938    -0.12803     0.066035    0.097801
 -0.016806    0.015525    0.084922   -0.23528    -0.017823    0.4899
 -0.15586    -0.14952    -0.30836     0.12642     0.10689     0.10892
 -0.17307     0.36345     0.10455     0.60661    -0.33281    -0.18994
  0.20595     0.074113    0.1325     -0.30423     0.076651    0.18055
 -0.29682    -0.17862    -0.43857    -0.002312    0.47742     0.096643
 -0.32785    -0.067572    0.013655   -0.20559     0.49405    -0.18791

In [5]:
#ai préféré utiliser le tokenizer de keras plutôt que celui de nltk car les valeurs filtrées par défaut comprennent l'apostrophe
#def tokenize(sentence):
#	tokens = text_to_word_sequence(sentence)
#	return tokens
	
#dataset['tokens']= dataset['text'].map(tokenize)
#dataset['tokens']

MAX_NUM_WORDS = 20000
MAX_SEQUENCE_LENGTH = 5
texts = dataset['text']
# finally, vectorize the text samples into a 2D integer tensor
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

#labels = to_categorical(np.asarray(labels))
print('Shape of data tensor:', data.shape)
#print('Shape of label tensor:', labels.shape)
dataset['tokens']= list(data)
dataset

Found 155 unique tokens.
Shape of data tensor: (139, 5)


,text,class,tokens
0,à la bonne franquette,éco,"[0, 25, 15, 65, 66]"
1,à prix modique,éco,"[0, 0, 25, 11, 34]"
2,aucun problème d'argent,cher,"[0, 0, 67, 68, 69]"
3,avantageux,éco,"[0, 0, 0, 0, 70]"
4,bas de gamme,éco,"[0, 0, 26, 4, 12]"
5,bas prix,éco,"[0, 0, 0, 26, 11]"
6,bistrot pas cher,éco,"[0, 0, 71, 2, 3]"
7,bon,cher,"[0, 0, 0, 0, 6]"
8,bon japonais,cher,"[0, 0, 0, 6, 35]"
9,bon marché,éco,"[0, 0, 0, 6, 27]"


# Préparer la couche d'intégration ( the Embedding layer)

Il s'agit de construire une matrice (nombre de mots x vecteur de 300) qui permette de retrouver le vecteur de chacun des mots du vocabulaire et dinitialiser la couche d'intégration.

Un des problèmes rencontré est que faire lorsque le token ne fait pas parti des mots pré-vectorisés.

PLusieurs solutions étainet possibles on est parti pour attrapper l'erreur en cas de mot ou-of-the-vocabulary en le remplacant par le mot inconnu.

Il restera à tester d'autres familles de wrapper de textfast qui malgré que le mot soit ou-of-vacubalary arrive à lui générer un vecteur, il faudra comprendre lequel et pourquoi !!.



In [6]:
# prepare embedding matrix
EMBEDDING_DIM = 300
num_words = min(MAX_NUM_WORDS, len(word_index))
print(num_words)
embedding_matrix = np.zeros((num_words + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    # words not found in embedding index will be all-zeros.
    print (word,i)
    try:
        embedding_vector = fr_model[word]
    except KeyError:
        # word not found 
        embedding_vector = embedding_vector
    embedding_matrix[i]=embedding_vector
      


print(embedding_matrix.shape)
embedding_matrix



155
un 1
pas 2
cher 3
de 4
très 5
bon 6
restaurant 7
je 8
moins 9
plutôt 10
prix 11
gamme 12
euros 13
vingt 14
la 15
qualité 16
pour 17
du 18
le 19
des 20
bien 21
peu 22
restau 23
vraiment 24
à 25
bas 26
marché 27
burger 28
cent 29
et 30
haut 31
fiche 32
truc 33
modique 34
japonais 35
classe 36
cantine 37
deux 38
dix 39
économique 40
entre 41
trente 42
les 43
j'aime 44
m’en 45
m'en 46
luxueux 47
menu 48
modeste 49
quinze 50
moyen 51
ni 52
tout 53
trop 54
routier 55
petit 56
troquet 57
plus 58
rapide 59
standing 60
super 61
grand 62
produits 63
avec 64
bonne 65
franquette 66
aucun 67
problème 68
d'argent 69
avantageux 70
bistrot 71
rapport 72
caviar 73
ce 74
n'est 75
important 76
c'est 77
fin 78
mois 79
éco 80
10 81
20 82
frites 83
gastronomique 84
j’ai 85
moyens 86
king 87
œufs 88
fous 89
n'ai 90
sou 91
suis 92
riche 93
jusqu'à 94
cinquante 95
kebab 96
meilleur 97
meilleurs 98
luxe 99
ma 100
bourse 101
est 102
pleine 103
mac 104
do 105
maccaroni 106
midi 107
chic 108
trois 109
étoiles 

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.19326   , -0.023721  ,  0.0096097 , ...,  0.22441   ,
        -0.20043001,  0.15966   ],
       [ 0.087169  , -0.042794  , -0.044115  , ...,  0.27595001,
         0.074523  ,  0.16216999],
       ...,
       [-0.22148   ,  0.019692  , -0.08569   , ..., -0.17587   ,
         0.21257   ,  0.052583  ],
       [-0.035705  ,  0.42269   , -0.52911001, ...,  0.1505    ,
         0.15895   , -0.060457  ],
       [-0.33131   , -0.11218   , -0.20186999, ...,  0.25556999,
         0.17278001,  0.22938   ]])

In [7]:
# load pre-trained word embeddings into an Embedding layer
# note that we set trainable = False so as to keep the embeddings fixed
embedding_layer = Embedding(num_words + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

Le prochain pavé permet de transformer les catégories "éco","moyen" et cher en vecteur.
Ces vecteurs sont rangés dans la colonne 'Ycodes' du dataset

In [8]:
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)
dataset['Ycodes'] = pd.Series(list(dummy_y))
dataset


,text,class,tokens,Ycodes
0,à la bonne franquette,éco,"[0, 25, 15, 65, 66]","[0.0, 0.0, 0.0, 1.0]"
1,à prix modique,éco,"[0, 0, 25, 11, 34]","[0.0, 0.0, 0.0, 1.0]"
2,aucun problème d'argent,cher,"[0, 0, 67, 68, 69]","[1.0, 0.0, 0.0, 0.0]"
3,avantageux,éco,"[0, 0, 0, 0, 70]","[0.0, 0.0, 0.0, 1.0]"
4,bas de gamme,éco,"[0, 0, 26, 4, 12]","[0.0, 0.0, 0.0, 1.0]"
5,bas prix,éco,"[0, 0, 0, 26, 11]","[0.0, 0.0, 0.0, 1.0]"
6,bistrot pas cher,éco,"[0, 0, 71, 2, 3]","[0.0, 0.0, 0.0, 1.0]"
7,bon,cher,"[0, 0, 0, 0, 6]","[1.0, 0.0, 0.0, 0.0]"
8,bon japonais,cher,"[0, 0, 0, 6, 35]","[1.0, 0.0, 0.0, 0.0]"
9,bon marché,éco,"[0, 0, 0, 6, 27]","[0.0, 0.0, 0.0, 1.0]"


###MODEL DEFINITION
Le réseau comprend trois couches.

Chaque phrase est représentée à l'aide d'un vecteur de longueur cinq à l'aide d'un vocabulaire à neuf mots.

In [9]:
########################### MODEL DEFINITION
# train a 1D convnet with global maxpooling
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
x = Conv1D(128, 5, activation='relu')(embedded_sequences)
x = MaxPooling1D(1)(x)
x = Conv1D(128, 1, activation='relu')(x)
x = MaxPooling1D(1)(x)
x = Conv1D(128, 1, activation='relu')(x)
x = GlobalMaxPooling1D()(x)
x = Dense(128, activation='relu')(x)
preds = Dense(3, activation='softmax')(x)



In [10]:
model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])



In [11]:
x_train = data
y_train = dummy_y

model.fit(x_train, y_train,
          batch_size=128,
          epochs=20)
#          validation_data=(x_val, y_val))
    


ValueError: Error when checking target: expected dense_2 to have shape (3,) but got array with shape (4,)

In [ ]:
###### TRAINING + EVALUATION
x_test = x_train
y_test = y_train
loss,metric = model.evaluate(x_test, y_test)
print ("Loss = ", loss," Accuracy = ",metric)

In [ ]:
dataset['predicted'] = list(model.predict(x_train)) 

def decodeY(L):
	if L[0]>L[1] and L[0]>L[2]: 
		return "cher"
	elif L[2]>L[1] and L[2]>L[0]:
		return "éco"
	else:
		return "moyen"
	
dataset['resultat']=list(map(decodeY,dataset['predicted']))
dataset


In [ ]:
# split the data into a training set and a validation set
VALIDATION_SPLIT = 0.2
labels = dummy_y
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
num_validation_samples = int(VALIDATION_SPLIT * data.shape[0])
x_train = data[:-num_validation_samples]
y_train = labels[:-num_validation_samples]
x_val = data[-num_validation_samples:]
y_val = labels[-num_validation_samples:]



In [ ]:
model.fit(x_train, y_train,
          batch_size=128,
          epochs=50,
          validation_data=(x_val, y_val))


In [ ]:
labels


In [ ]:
dataset['predicted'] = list(model.predict(data)) 

def decodeY(L):
	if L[0]>L[1] and L[0]>L[2]: 
		return "cher"
	elif L[2]>L[1] and L[2]>L[0]:
		return "éco"
	else:
		return "moyen"
	
    

dataset['resultat']=list(map(decodeY,dataset['predicted']))
dataset


In [ ]:
import matplotlib
import matplotlib.pyplot as plt


for x,y,z in dataset['predicted']:
    #print(x,z)
    plt.plot(x,z,'ro')


#plt.plot([1,2,3,4], [1,4,9,16], 'ro')
plt.axis([-0.1, 1.1, -0.1, 1.1])
plt.show()

In [ ]:
while True:
    inp = input("Tu préfères aller dans un restau plutôt cher ou plutôt pas cher ?(fin pour sortir)")
    if (inp == 'fin'):
        break
    sequences = tokenizer.texts_to_sequences([inp])
    padded_sequence = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
    prediction = model.predict(padded_sequence)
    print(prediction)
    #decodeY(model.predict(pad_sequences(tokenizer.texts_to_sequences(['pas cher']), maxlen=MAX_SEQUENCE_LENGTH))[0]))
    print("ça veut dire ->" ,decodeY(prediction[0]))
    


#### 
